In [1]:
# unique/distinct data 
# window function 
# databricks community cloud 

In [2]:
from pyspark.sql import SparkSession


spark = (
    SparkSession
    .builder
    .appName("spark introduction")
    .master("local[*]")
    .getOrCreate()
)

In [3]:
# Updated schema with gender and hire_date
emp_schema = """
emp_id int,
emp_name string,
gender string,
department string,
role string,
salary int,
hire_date string
"""

# Expanded employee data with hire_date
emp_data1 = [
    [101, "Alice", "Female", "HR", "Manager", 60000, "2018-03-15"],
    [102, "Bob", "Male", "IT", "Developer", 75000, "2019-07-22"],
    [103, "Charlie", "Male", "Finance", "Analyst", 65000, "2020-01-10"],
    [104, "Diana", "Female", "IT", "Tester", 55000, "2021-05-18"],
    [105, "Evan", "Male", "Sales", "Executive", 50000, "2022-02-01"],
    [106, "Fiona", "Female", "Marketing", "Coordinator", 48000, "2020-09-12"],
    [107, "George", "", "IT", "DevOps Engineer", 82000, "2017-11-30"],
    [108, "Hannah", "Female", "Finance", "Accountant", 62000, "2019-04-25"],
    [109, "Ian", "Male", "HR", "Recruiter", 52000, "2021-08-09"],
    [110, "Julia", "Female", "Sales", "Manager", 70000, "2016-06-17"]
]


emp_data2 = [
    [101, "Alan", "Female", "HR", "Manager", 60000, "2018-03-15"],
    [102, "Bobmark", "Male", "IT", "Developer", 75000, "2019-07-22"],
    [103, "Cheery", "Male", "Finance", "Analyst", 65000, "2020-01-10"],
    [104, "Direy", "Female", "IT", "Tester", 55000, "2021-05-18"],
    [105, "Egks", "Male", "Sales", "Executive", 50000, "2022-02-01"],
    [106, "Fatima", "Female", "Marketing", "Coordinator", 48000, "2020-09-12"],
    [107, "Gogg", "", "IT", "DevOps Engineer", 82000, "2017-11-30"],
    [108, "Hatim", "Female", "Finance", "Accountant", 62000, "2019-04-25"],
    [109, "Ismile", "Male", "HR", "Recruiter", 52000, "2021-08-09"],
    [110, "Jafari", "Female", "Sales", "Manager", 70000, "2016-06-17"]
]




In [4]:
emp1 = spark.createDataFrame(data=emp_data1,schema=emp_schema)
emp2 = spark.createDataFrame(data=emp_data2,schema=emp_schema)


In [5]:
emp2other = emp2.select("emp_id","gender","salary","emp_name","department","role","hire_date")

In [6]:
emp = emp1.unionByName(emp2other)

In [7]:
emp.printSchema()

root
 |-- emp_id: integer (nullable = true)
 |-- emp_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- department: string (nullable = true)
 |-- role: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- hire_date: string (nullable = true)



In [8]:
# unique data get 
# select distinct emp * from emp 

emp_unique = emp.distinct()

In [9]:
emp_unique.show()

+------+--------+------+----------+---------------+------+----------+
|emp_id|emp_name|gender|department|           role|salary| hire_date|
+------+--------+------+----------+---------------+------+----------+
|   101|   Alice|Female|        HR|        Manager| 60000|2018-03-15|
|   102|     Bob|  Male|        IT|      Developer| 75000|2019-07-22|
|   103| Charlie|  Male|   Finance|        Analyst| 65000|2020-01-10|
|   104|   Diana|Female|        IT|         Tester| 55000|2021-05-18|
|   105|    Evan|  Male|     Sales|      Executive| 50000|2022-02-01|
|   106|   Fiona|Female| Marketing|    Coordinator| 48000|2020-09-12|
|   107|  George|      |        IT|DevOps Engineer| 82000|2017-11-30|
|   108|  Hannah|Female|   Finance|     Accountant| 62000|2019-04-25|
|   109|     Ian|  Male|        HR|      Recruiter| 52000|2021-08-09|
|   110|   Julia|Female|     Sales|        Manager| 70000|2016-06-17|
|   101|    Alan|Female|        HR|        Manager| 60000|2018-03-15|
|   102| Bobmark|  M

In [10]:
# unique of department ids 

emp_dept_id = emp.select("emp_id").distinct()

In [11]:
emp_dept_id.show()

+------+
|emp_id|
+------+
|   101|
|   102|
|   103|
|   105|
|   104|
|   106|
|   107|
|   108|
|   109|
|   110|
+------+



In [18]:
# window function or analytical function 
# applies aggregate and ranking function 
# over a particular window set of rows 

from pyspark.sql.window import Window
from pyspark.sql.functions import max,col,desc


window_spec = Window.partitionBy(col("department")).orderBy(col("salary").desc())
max_func = max(col("salary")).over(window_spec)


In [19]:
emp_1 = emp.withColumn("max_salary",max_func)

In [23]:
emp_1.show(1)

+------+--------+------+----------+-------+------+----------+----------+
|emp_id|emp_name|gender|department|   role|salary| hire_date|max_salary|
+------+--------+------+----------+-------+------+----------+----------+
|   103| Charlie|  Male|   Finance|Analyst| 65000|2020-01-10|     65000|
+------+--------+------+----------+-------+------+----------+----------+
only showing top 1 row



In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,desc,col

window_spec = Window.partitionBy(col("role")).orderBy(col("salary").desc())

rn = row_number().over(window_spec)

emp_2= emp.withColumn("rn",rn).where("rn = 2")

In [29]:
emp_2.show()

+------+--------+------+----------+---------------+------+----------+---+
|emp_id|emp_name|gender|department|           role|salary| hire_date| rn|
+------+--------+------+----------+---------------+------+----------+---+
|   108|   Hatim|Female|   Finance|     Accountant| 62000|2019-04-25|  2|
|   103|  Cheery|  Male|   Finance|        Analyst| 65000|2020-01-10|  2|
|   106|  Fatima|Female| Marketing|    Coordinator| 48000|2020-09-12|  2|
|   107|    Gogg|      |        IT|DevOps Engineer| 82000|2017-11-30|  2|
|   102| Bobmark|  Male|        IT|      Developer| 75000|2019-07-22|  2|
|   105|    Egks|  Male|     Sales|      Executive| 50000|2022-02-01|  2|
|   110|  Jafari|Female|     Sales|        Manager| 70000|2016-06-17|  2|
|   109|  Ismile|  Male|        HR|      Recruiter| 52000|2021-08-09|  2|
|   104|   Direy|Female|        IT|         Tester| 55000|2021-05-18|  2|
+------+--------+------+----------+---------------+------+----------+---+



In [33]:
from pyspark.sql.functions import expr

emp_3 = emp.withColumn("rn",expr("row_number() over (partition by role order by salary desc)")).where("rn = 2")

In [34]:
emp_3.show()

+------+--------+------+----------+---------------+------+----------+---+
|emp_id|emp_name|gender|department|           role|salary| hire_date| rn|
+------+--------+------+----------+---------------+------+----------+---+
|   108|   Hatim|Female|   Finance|     Accountant| 62000|2019-04-25|  2|
|   103|  Cheery|  Male|   Finance|        Analyst| 65000|2020-01-10|  2|
|   106|  Fatima|Female| Marketing|    Coordinator| 48000|2020-09-12|  2|
|   107|    Gogg|      |        IT|DevOps Engineer| 82000|2017-11-30|  2|
|   102| Bobmark|  Male|        IT|      Developer| 75000|2019-07-22|  2|
|   105|    Egks|  Male|     Sales|      Executive| 50000|2022-02-01|  2|
|   110|  Jafari|Female|     Sales|        Manager| 70000|2016-06-17|  2|
|   109|  Ismile|  Male|        HR|      Recruiter| 52000|2021-08-09|  2|
|   104|   Direy|Female|        IT|         Tester| 55000|2021-05-18|  2|
+------+--------+------+----------+---------------+------+----------+---+



In [ ]:
# using data brick community you can create free compute 